In [1]:
import ipyparallel as ipp
print(ipp.version_info)
from IPython.display import display

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids


(8, 6, 1)
<Cluster(cluster_id='1686992415-fxfy', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.42s/engine]


[0, 1, 2, 3]

In [2]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1686992415-fxfy                  True          4 MPI


In [3]:
%autopx

%autopx enabled


In [5]:
import ipyparallel as ipp

import numpy as np
from mpi4py import MPI
from mpi4py.MPI import IN_PLACE, SUM
from mpi4py_fft import DistArray



class Test:

    shape = (10, 10)

    def __init__(self, val):
        self.buffer = np.empty(self.shape, dtype='i8')
        self.buffer[:] = val

        
check = None
comm = MPI.COMM_WORLD
comm_size, comm_rank = comm.Get_size(), comm.Get_rank()

arr = Test(comm_rank)
print(comm_size)
check = None
if comm_rank == 1:
    raise Exception('an error')
val_final = (comm_size - 1) * comm_size // 2
comm.Allreduce(IN_PLACE, arr.buffer, op=SUM)
check = np.all(arr.buffer == val_final)

print(f"{comm_rank}/{comm_size}: {check}")

[stdout:1] 4


[1:execute]
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[2], line 27
     25 check = None
     26 if comm_rank == 1:
---> 27     raise Exception('an error')
     28 val_final = (comm_size - 1) * comm_size // 2
     29 comm.Allreduce(IN_PLACE, arr.buffer, op=SUM)

Exception: an error


%px:  25%|███████████████████████████████████████████████████████▊                                                                                                                                                                       | 1/4 [00:05<00:17,  5.87s/tasks]


Received Keyboard Interrupt. Sending signal SIGINT to engines...


In [7]:
print(check)

[stdout:1] None


%px:  25%|███████████████████████████████████████████████████████▊                                                                                                                                                                       | 1/4 [00:04<00:14,  4.84s/tasks]


Received Keyboard Interrupt. Sending signal SIGINT to engines...


Controller stopped: {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 13983, 'identifier': 'ipcontroller-1686992415-fxfy-13968'}
engine set stopped 1686992416: {'exit_code': <Negsignal.SIGKILL: -9>, 'pid': 14012, 'identifier': 'ipengine-1686992415-fxfy-1686992416-13968'}


In [9]:
import ipyparallel as ipp

import numpy as np
from mpi4py import MPI
from mpi4py.MPI import IN_PLACE, SUM
from mpi4py_fft import DistArray

from qtmpy.mpi.comm import CommMod



class Test:

    shape = (10, 10)

    def __init__(self, val):
        self.buffer = np.empty(self.shape, dtype='i8')
        self.buffer[:] = val

        
check = None
with CommMod(MPI.COMM_WORLD) as comm:
    comm_size, comm_rank = comm.Get_size(), comm.Get_rank()

    arr = Test(comm_rank)
    print(comm_size)
    val_final = (comm_size - 1) * comm_size // 2
    comm.Allreduce_sum_inplace(arr.buffer)
    check = np.all(arr.buffer == val_final)

print(f"{comm_rank}/{comm_size}: {check}")

[stdout:1] 4
1/4: True


[stdout:0] 4
0/4: True


[stdout:2] 4
2/4: True


[stdout:3] 4
3/4: True


In [4]:
%%px
import ipyparallel as ipp

import numpy as np
from mpi4py import MPI
from mpi4py.MPI import IN_PLACE, SUM
from mpi4py_fft import DistArray

from qtmpy.mpi.comm import CommMod



class Test:

    shape = (10, 10)

    def __init__(self, val):
        self.buffer = np.empty(self.shape, dtype='i8')
        self.buffer[:] = val

        
check = None
with CommMod(MPI.COMM_WORLD) as comm:
    comm_size, comm_rank = comm.Get_size(), comm.Get_rank()

    arr = Test(comm_rank)
    print(comm_size)
    if comm_rank == 1:
        print(1/0)
    val_final = (comm_size - 1) * comm_size // 2
    comm.Allreduce_sum_inplace(arr.buffer)
    check = np.all(arr.buffer == val_final)

print(f"{comm_rank}/{comm_size}: {check}")

[stdout:1] 4


[1:execute]
---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
File ~/qtmpy/src/qtmpy/mpi/comm.py:106, in CommMod.__exit__(***failed resolving arguments***)
    105     if isinstance(exc_val, Exception):
--> 106         raise exc_val
    107 except Exception as exc_val:

Cell In[1], line 28
     27 if comm_rank == 1:
---> 28     print(1/0)
     29 val_final = (comm_size - 1) * comm_size // 2

ZeroDivisionError: division by zero

The above exception was the direct cause of the following exception:

Exception                                 Traceback (most recent call last)
Cell In[1], line 22
     18         self.buffer[:] = val
     21 check = None
---> 22 with CommMod(MPI.COMM_WORLD) as comm:
     23     comm_size, comm_rank = comm.Get_size(), comm.Get_rank()
     25     arr = Test(comm_rank)

File ~/qtmpy/src/qtmpy/mpi/comm.py:108, in CommMod.__exit__(***failed resolving arguments***)

[stdout:2] 4


[stdout:3] 4


[stdout:0] 4


%px: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.44tasks/s]


AlreadyDisplayedError: 4 errors

In [5]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1686991996-vd1q                  True          0 MPI


In [6]:
cluster.restart_engines_sync()

Stopping engine(s): 1686991997
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


In [7]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1686991996-vd1q                  True          4 MPI
